In [1]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

ROM_SIZE = 0x2000 #8K

SOC_UP = 0x0000;
SOC_LA = 0x1000;
PL_AA_MB = 0x2000;
PL_AA = 0x2100;
SOC_IS = 0x3000;
SOC_AS = 0x4000;
SOC_CC = 0x5000;
PL_AS = 0x6000;
PL_IS = 0x7000;
PL_DMA = 0x8000;
ol = Overlay("design_1.bit")
#ol.ip_dict
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
# ipReadROMCODE = ol.read_romcode_0

#Add for SPI
ip_QSPI = ol.axi_quad_spi_0
# ============================================
# AXI QuadSPI Control
# ============================================
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04
XSP_CR_CLK_POLARITY_MASK = 0x08
XSP_CR_CLK_PHASE_MASK = 0x10
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def write_tx_fifo(AxiQspi):
    #print("TransferData")
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    StatusReg = AxiQspi.read(XSP_SR_OFFSET)
    while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
        StatusReg = AxiQspi.read(XSP_SR_OFFSET)

    #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)


def read_rx_fifo(bypass_length, AxiQspi):
    #print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    
    # By pass the FIFO data during master issue command and address to slave device
    command_addr_length = bypass_length
    counter = 0    
    
    while RxFifoStatus == 0:
        #temp = AxiQspi.read(XSP_RFO_OFFSET)
        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        #print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    

        counter = counter + 1
        if(counter > command_addr_length):
            resp.append(temp)        
        
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))
# ============================================
# Release Reset First before passthrough mode
# ============================================
# Release Caravel reset
# 0x10 : Data signal of outpin_ctrl
#        bit 0  - outpin_ctrl[0] (Read/Write)
#        others - reserved
print (ipOUTPIN.read(0x10))
ipOUTPIN.write(0x10, 1)
print (ipOUTPIN.read(0x10))
# ============================================
# Load firmware (fsic.hex) to memory npROM
# ============================================

# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)

npROM_index = 0
npROM_offset = 0
fiROM = open("fsic.hex", "r+")

for line in fiROM:
    # offset header
    if line.startswith('@'):
        # Ignore first char @
        npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
        npROM_offset = npROM_offset >> 2 # 4byte per offset
        #print (npROM_offset)
        npROM_index = 0
        continue
    #print (line)

    # We suppose the data must be 32bit alignment
    buffer = 0
    bytecount = 0
    for line_byte in line.strip(b'\x00'.decode()).split():
        buffer += int(line_byte, base = 16) << (8 * bytecount)
        bytecount += 1
        # Collect 4 bytes, write to npROM
        if(bytecount == 4):
            npROM[npROM_offset + npROM_index] = buffer
            # Clear buffer and bytecount
            buffer = 0
            bytecount = 0
            npROM_index += 1
            #print (npROM_index)
            continue
    # Fill rest data if not alignment 4 bytes
    if (bytecount != 0):
        npROM[npROM_offset + npROM_index] = buffer
        npROM_index += 1
    
fiROM.close()
# ============================================
# Enabling passthrou mode
# ============================================
cnfg(ip_QSPI)
# Passthrou mode - Write command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x02) # Command: Write data to memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
write_tx_fifo(ip_QSPI)

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
# ============================================
# Writing FW into SPIROM
# ============================================
# Fill up Tx_FIFO (16) for each write_tx_fifo
for index in range (ROM_SIZE >> 2):
     # 4 bytes alignment in npROM
    for byte_shift in range(4):
        tmp = int((npROM[index] >> (byte_shift * 8)) & 0xFF)
        ip_QSPI.write(XSP_DTR_OFFSET, tmp) # Write_data
    # TX_FIFO = 16, 4 * 4 = 16
    if((index % 3) == 3):
        write_tx_fifo(ip_QSPI)
        
# If rest data is not enough 16 bytes. Tx_FIFO is not empty
    StatusReg = ip_QSPI.read(XSP_SR_OFFSET)
    if ((StatusReg & XSP_SR_TX_EMPTY_MASK) == 0):
         write_tx_fifo(ip_QSPI)
# ============================================
# Read SPIROM for testing
# ============================================
cnfg(ip_QSPI)
# Test Passthrou mode - Read command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x03) # Command: Read data from memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(5, ip_QSPI)
for data in rx_final:
    print (hex(data))
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(0, ip_QSPI)
for data in rx_final:
    print (hex(data))
# ============================================
# Exit passthrou mode, FW will be fetched
# ============================================
ip_QSPI.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))
IP_BASE_ADDRESS = 0x60000000
ADDRESS_RANGE = 0xa000
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
# PL_IS Config
ADDRESS_OFFSET = PL_IS #0x7000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345671)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345673)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# PL_DMA Config
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))
# PL_AS Config
ADDRESS_OFFSET = PL_AS # 0x6000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345676)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# PL_AA Config
ADDRESS_OFFSET = PL_AA # 0x2100
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04))) 
mmio.write(ADDRESS_OFFSET+0x00, 0x11111111)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222222)
# PL_AA_MB Mailbox
ADDRESS_OFFSET = PL_AA_MB # 0x2000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))
print("mmio.read(ADDRESS_OFFSET+0x08): ", hex(mmio.read(ADDRESS_OFFSET+0x08)))
print("mmio.read(ADDRESS_OFFSET+0x0C): ", hex(mmio.read(ADDRESS_OFFSET+0x0C)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x14): ", hex(mmio.read(ADDRESS_OFFSET+0x14)))
print("mmio.read(ADDRESS_OFFSET+0x18): ", hex(mmio.read(ADDRESS_OFFSET+0x18)))
print("mmio.read(ADDRESS_OFFSET+0x1C): ", hex(mmio.read(ADDRESS_OFFSET+0x1C)))
mmio.write(ADDRESS_OFFSET, 0x11111112)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222223)
mmio.write(ADDRESS_OFFSET+0x08, 0x33333334)
mmio.write(ADDRESS_OFFSET+0x0C, 0x44444445)
mmio.write(ADDRESS_OFFSET+0x10, 0x55555556)
mmio.write(ADDRESS_OFFSET+0x14, 0x66666667)
mmio.write(ADDRESS_OFFSET+0x18, 0x77777778)
mmio.write(ADDRESS_OFFSET+0x1C, 0x88888889)
# Caravel-IS Config
ADDRESS_OFFSET = SOC_IS # 0x3000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# Caravel-IS Config
ADDRESS_OFFSET = 0x3000
mmio.write(ADDRESS_OFFSET, 0x00000003)
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
mmio.write(ADDRESS_OFFSET, 0x00000006)
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000)
# Caravel-LA Config
ADDRESS_OFFSET = SOC_LA # 0x1000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# Caravel-UP Config
ADDRESS_OFFSET = SOC_UP # 0x0000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))

0x10 =  0x0
0x14 =  0x0
0x1c =  0x0
0x20 =  0x0
0x34 =  0xfffffff7
0x38 =  0x3f
0
1
Configure device
XSP_TFO_OFFSET  : 0x00000004
XSP_TFO_OFFSET  : 0x00000000
Configure device
XSP_TFO_OFFSET  : 0x0000000c
0x6f
0x0
0x0
0xb
0x13
0x0
0x0
0x0
XSP_TFO_OFFSET  : 0x00000007
0x13
0x0
0x0
0x0
0x13
0x0
0x0
0x0
0x10 =  0x0
0x14 =  0x0
0x1c =  0x0
0x20 =  0x0
0x34 =  0x3ffff6
0x38 =  0x10
mmio.read(ADDRESS_OFFSET):  0x0
mmio.read(ADDRESS_OFFSET):  0x1
mmio.read(ADDRESS_OFFSET):  0x3
mmio.read(ADDRESS_OFFSET):  0x4
mmio.read(ADDRESS_OFFSET+0x10):  0x0
mmio.read(ADDRESS_OFFSET+0x20):  0x0
mmio.read(ADDRESS_OFFSET+0x28):  0x0
mmio.read(ADDRESS_OFFSET+0x30):  0x0
mmio.read(ADDRESS_OFFSET+0x38):  0x0
mmio.read(ADDRESS_OFFSET+0x3C):  0x0
mmio.read(ADDRESS_OFFSET):  0x6
mmio.read(ADDRESS_OFFSET):  0x6
mmio.read(ADDRESS_OFFSET+0x00):  0x0
mmio.read(ADDRESS_OFFSET+0x04):  0x0
mmio.read(ADDRESS_OFFSET):  0x0
mmio.read(ADDRESS_OFFSET+0x04):  0x0
mmio.read(ADDRESS_OFFSET+0x08):  0x0
mmio.read(ADDRESS_OFFSET+0

In [2]:
# choose uprj1
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000002)

In [3]:
#setting dma 
mmio_usedma = MMIO(0x40020000, 0x10000)

In [5]:
# NTT
userma_buf_out= allocate(shape=(1024,), dtype=np.uint64)
mmio_usedma.write(0x38, userma_buf_out.device_address)  # output buffer addr low
mmio_usedma.write(0x3c, 0x00000000) # outputbuffer addr high
userma_buf= allocate(shape=(1024,), dtype=np.uint64)
mmio_usedma.write(0x44,userma_buf.device_address)  # input buffer addr low
mmio_usedma.write(0x48, 0x00000000) # inputbuffer addr high
mmio_usedma.write(0x10, 0x0000002) # mode

for i in range(1024):
    userma_buf[i]=i

# dma_start
mmio_usedma.write(0x00,0x00000001) 
timeKernelStart = time()
while True:
    if mmio_usedma.read(0x18) == 0x01:
        break
timeKernelEnd = time()

# FFT output
for i in range(1024):
    print(userma_buf_out[i])

# Execution time
print("NTT execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
print(hex(userma_buf.device_address-userma_buf_out.device_address))

11436
2816
10436
2523
6525
9420
6666
11263
3079
6734
4705
9067
1032
10817
4531
10967
3186
11171
7805
10868
10392
8004
4575
6794
338
8738
5120
3541
10471
6944
7558
9251
8276
4997
10111
7051
4738
8769
9227
4258
10308
9229
9706
10006
4004
7149
7972
9341
11639
10833
10122
4401
11328
5299
4239
3367
7123
1935
9219
6199
6036
7904
11446
4186
1714
6060
3320
482
4365
6781
2712
1839
686
5415
5631
12222
10984
281
896
2078
7509
11293
11264
10054
4737
9939
7009
1240
9316
9117
6024
7582
4797
5281
3194
11199
4524
11711
10413
5764
10657
10179
8710
4178
5544
2027
4186
8600
4522
1540
9629
12161
1015
2570
1
5580
4601
9937
5844
4731
11628
4431
2722
8190
8686
7360
3590
10524
1639
7113
9039
11697
9300
4641
9365
9580
9658
9214
1597
8864
12185
4643
9117
10773
9898
2566
7447
6744
7312
5970
12042
1282
10400
11928
2802
9578
7631
4659
8113
7257
11691
8312
5221
10396
4111
4834
8025
6750
4631
2724
328
11970
8872
8578
12141
794
434
345
3412
2525
3766
925
3950
424
6624
10784
7627
5942
3294
8632
7525
6693
8260
4987
230

In [ ]:
'''
FFT_in = [5856.000000, 
9672.000000, 354.000000, 9719.000000, 7445.000000, 6283.000000, 10462.000000, 8588.000000, 85.000000, 3840.000000, 154.000000, 4853.000000, 9810.000000, 2498.000000, 6289.000000, 11201.000000, 10788.000000, 1348.000000, 4887.000000, 9929.000000, 6350.000000, 11349.000000, 9022.000000, 8339.000000, 1628.000000, 6696.000000, 7021.000000, 7449.000000, 11889.000000, 6671.000000, 6476.000000, 
8912.000000, 5472.000000, 9103.000000, 11621.000000, 4985.000000, 11799.000000, 7779.000000, 6236.000000, 4880.000000, 3469.000000, 2493.000000, 455.000000, 8968.000000, 7869.000000, 9871.000000, 3795.000000, 10502.000000, 24.000000, 9231.000000, 1668.000000, 10999.000000, 5937.000000, 10579.000000, 6780.000000, 1407.000000, 149.000000, 922.000000, 884.000000, 5092.000000, 7485.000000, 
10910.000000, 3231.000000, 9806.000000, 9181.000000, 4821.000000, 7829.000000, 10197.000000, 9857.000000, 1682.000000, 5559.000000, 9831.000000, 10176.000000, 3765.000000, 10055.000000, 12080.000000, 5137.000000, 11767.000000, 7822.000000, 2386.000000, 7572.000000, 7172.000000, 2543.000000, 3735.000000, 5842.000000, 9635.000000, 6402.000000, 11416.000000, 8387.000000, 9006.000000, 9905.000000, 
6464.000000, 7783.000000, 8087.000000, 469.000000, 5723.000000, 8863.000000, 1320.000000, 4171.000000, 4942.000000, 8036.000000, 7169.000000, 1063.000000, 1414.000000, 4290.000000, 4015.000000, 2886.000000, 5520.000000, 7406.000000, 6283.000000, 1727.000000, 757.000000, 2589.000000, 5012.000000, 10398.000000, 347.000000, 2945.000000, 4993.000000, 3535.000000, 12102.000000, 4117.000000, 
10037.000000, 9469.000000, 3593.000000, 607.000000, 6667.000000, 11300.000000, 2911.000000, 5494.000000, 10477.000000, 4953.000000, 5479.000000, 8012.000000, 6745.000000, 4327.000000, 7739.000000, 4946.000000, 1830.000000, 8740.000000, 11249.000000, 10418.000000, 1337.000000, 2159.000000, 6103.000000, 10419.000000, 6852.000000, 10918.000000, 634.000000, 2147.000000, 11027.000000, 115.000000, 
11934.000000, 2008.000000, 5283.000000, 9696.000000, 5339.000000, 8111.000000, 10592.000000, 4456.000000, 3920.000000, 11139.000000, 7080.000000, 6830.000000, 275.000000, 11516.000000, 7928.000000, 8530.000000, 7695.000000, 1683.000000, 11501.000000, 5871.000000, 11165.000000, 7100.000000, 810.000000, 7857.000000, 5717.000000, 10166.000000, 9130.000000, 11194.000000, 10671.000000, 536.000000, 
737.000000, 12248.000000, 6353.000000, 8544.000000, 3937.000000, 5034.000000, 3917.000000, 8711.000000, 8462.000000, 10378.000000, 3165.000000, 4493.000000, 9024.000000, 2140.000000, 9035.000000, 7460.000000, 4719.000000, 3552.000000, 7015.000000, 4469.000000, 9174.000000, 5994.000000, 9150.000000, 6422.000000, 11130.000000, 3000.000000, 1976.000000, 8802.000000, 2033.000000, 11441.000000, 
1864.000000, 8543.000000, 611.000000, 2935.000000, 11438.000000, 2307.000000, 5860.000000, 5554.000000, 741.000000, 6519.000000, 8885.000000, 2954.000000, 5716.000000, 6994.000000, 3536.000000, 7164.000000, 5280.000000, 3461.000000, 6095.000000, 5498.000000, 11397.000000, 11592.000000, 9323.000000, 12119.000000, 5137.000000, 2923.000000, 5547.000000, 9278.000000, 6730.000000, 2513.000000, 
8428.000000, 10364.000000, 2322.000000, 12010.000000, 11719.000000, 4476.000000, 8565.000000, 1958.000000, 2343.000000, 2126.000000, 6759.000000, 6207.000000, 8633.000000, 9496.000000, 5827.000000, 5339.000000, 2665.000000, 4457.000000, 12239.000000, 6369.000000, 1174.000000, 6305.000000, 10726.000000, 6062.000000, 1470.000000, 40.000000, 3572.000000, 5825.000000, 9256.000000, 3030.000000, 
8189.000000, 4495.000000, 10957.000000, 7556.000000, 10444.000000, 3929.000000, 3336.000000, 7719.000000, 7701.000000, 3920.000000, 3240.000000, 5384.000000, 6158.000000, 10377.000000, 9603.000000, 11575.000000, 11387.000000, 6515.000000, 383.000000, 10946.000000, 10338.000000, 10925.000000, 9516.000000, 2561.000000, 9248.000000, 11489.000000, 5989.000000, 5931.000000, 10860.000000, 5884.000000, 
9448.000000, 12029.000000, 4249.000000, 4721.000000, 11411.000000, 6516.000000, 919.000000, 9414.000000, 4021.000000, 2813.000000, 6433.000000, 4060.000000, 6229.000000, 9324.000000, 8396.000000, 3655.000000, 4038.000000, 11401.000000, 11391.000000, 6624.000000, 6965.000000, 1187.000000, 7778.000000, 9525.000000, 5290.000000, 4589.000000, 1770.000000, 9836.000000, 1847.000000, 5239.000000, 
11924.000000, 4889.000000, 8212.000000, 1707.000000, 5735.000000, 4974.000000, 2531.000000, 449.000000, 2768.000000, 12064.000000, 9859.000000, 10323.000000, 3459.000000, 4735.000000, 9661.000000, 384.000000, 1189.000000, 7548.000000, 5661.000000, 11844.000000, 9595.000000, 6749.000000, 1936.000000, 6998.000000, 11774.000000, 2877.000000, 3385.000000, 2207.000000, 9872.000000, 4350.000000, 
11077.000000, 9953.000000, 455.000000, 2552.000000, 8605.000000, 9946.000000, 6442.000000, 7246.000000, 3157.000000, 10849.000000, 4656.000000, 1667.000000, 9455.000000, 1555.000000, 8596.000000, 5495.000000, 888.000000, 8049.000000, 11028.000000, 3015.000000, 10324.000000, 2961.000000, 2816.000000, 9379.000000, 8914.000000, 5548.000000, 6966.000000, 8344.000000, 8972.000000, 8966.000000, 
224.000000, 6524.000000, 7962.000000, 7014.000000, 4269.000000, 11969.000000, 9655.000000, 6309.000000, 11034.000000, 2971.000000, 417.000000, 2194.000000, 9184.000000, 8810.000000, 2677.000000, 763.000000, 7385.000000, 8099.000000, 1985.000000, 11618.000000, 4343.000000, 4646.000000, 4330.000000, 2255.000000, 3333.000000, 4466.000000, 6455.000000, 7587.000000, 7483.000000, 6205.000000, 
4553.000000, 5597.000000, 3179.000000, 6977.000000, 6462.000000, 6934.000000, 7464.000000, 11732.000000, 9158.000000, 7192.000000, 4401.000000, 6097.000000, 11916.000000, 640.000000, 9047.000000, 1129.000000, 6235.000000, 7792.000000, 10154.000000, 7615.000000, 11712.000000, 4726.000000, 6950.000000, 4287.000000, 5438.000000, 3839.000000, 3522.000000, 8654.000000, 287.000000, 11696.000000, 
6806.000000, 2209.000000, 8790.000000, 8208.000000, 9387.000000, 7654.000000, 10043.000000, 11137.000000, 3879.000000, 11513.000000, 11394.000000, 335.000000, 3681.000000, 5862.000000, 5871.000000, 3232.000000, 91.000000, 6080.000000, 3237.000000, 4310.000000, 6144.000000, 12168.000000, 10215.000000, 8661.000000, 10679.000000, 10909.000000, 2373.000000, 7161.000000, 7496.000000, 11050.000000, 
6570.000000, 555.000000, 11779.000000, 1676.000000, 3613.000000, 8205.000000, 7877.000000, 2609.000000, 3911.000000, 11342.000000, 570.000000, 8790.000000, 11693.000000, 6020.000000, 2933.000000, 5761.000000, 7284.000000, 1002.000000, 8389.000000, 8784.000000, 2452.000000, 5762.000000, 3906.000000, 7918.000000, 4778.000000, 461.000000, 7072.000000, 5622.000000, 7193.000000, 11329.000000, 
3073.000000, 6858.000000, 3900.000000, 6287.000000, 11292.000000, 11080.000000, 6353.000000, 3629.000000, 8090.000000, 11688.000000, 9714.000000, 7511.000000, 2563.000000, 10715.000000, 2508.000000, 7801.000000, 2557.000000, 916.000000, 1295.000000, 6651.000000, 6418.000000, 10108.000000, 11217.000000, 9338.000000, 6737.000000, 10409.000000, 2959.000000, 1293.000000, 9572.000000, 12032.000000, 
10789.000000, 11653.000000, 10462.000000, 7187.000000, 8367.000000, 2531.000000, 12179.000000, 3863.000000, 4057.000000, 9816.000000, 734.000000, 3389.000000, 12096.000000, 7233.000000, 4960.000000, 4570.000000, 2658.000000, 820.000000, 6382.000000, 632.000000, 11023.000000, 7280.000000, 3435.000000, 2482.000000, 1913.000000, 7023.000000, 9533.000000, 10280.000000, 6081.000000, 3646.000000, 
12266.000000, 424.000000, 8733.000000, 4725.000000, 1539.000000, 6704.000000, 9764.000000, 6520.000000, 7244.000000, 3443.000000, 10494.000000, 11916.000000, 11944.000000, 3767.000000, 2789.000000, 5669.000000, 9859.000000, 641.000000, 443.000000, 11076.000000, 2536.000000, 3534.000000, 8390.000000, 11788.000000, 7339.000000, 5206.000000, 760.000000, 9294.000000, 9545.000000, 8756.000000, 
6034.000000, 9787.000000, 4303.000000, 1043.000000, 5575.000000, 860.000000, 2621.000000, 11280.000000, 12077.000000, 8035.000000, 5223.000000, 4583.000000, 7951.000000, 11222.000000, 10971.000000, 3537.000000, 7731.000000, 8094.000000, 5838.000000, 223.000000, 4275.000000, 9957.000000, 2084.000000, 5955.000000, 4011.000000, 7657.000000, 9173.000000, 3597.000000, 2712.000000, 10725.000000, 
10346.000000, 3651.000000, 350.000000, 3169.000000, 11400.000000, 1825.000000, 5049.000000, 7806.000000, 1173.000000, 11542.000000, 8543.000000, 7648.000000, 8199.000000, 4681.000000, 7054.000000, 1808.000000, 2798.000000, 2394.000000, 3944.000000, 5883.000000, 11843.000000, 1753.000000, 10760.000000, 8979.000000, 11827.000000, 6778.000000, 8584.000000, 6868.000000, 4747.000000, 6676.000000, 
5529.000000, 4707.000000, 9596.000000, 5927.000000, 11200.000000, 12274.000000, 6497.000000, 7278.000000, 9540.000000, 7538.000000, 2256.000000, 11887.000000, 5972.000000, 11673.000000, 10582.000000, 7963.000000, 9216.000000, 7581.000000, 10778.000000, 10853.000000, 3564.000000, 9854.000000, 10547.000000, 5543.000000, 11609.000000, 3186.000000, 2957.000000, 936.000000, 8315.000000, 4490.000000, 
7193.000000, 10426.000000, 6110.000000, 11637.000000, 7138.000000, 8593.000000, 7410.000000, 5410.000000, 3688.000000, 10817.000000, 7093.000000, 1293.000000, 6357.000000, 10232.000000, 10823.000000, 4042.000000, 5633.000000, 1375.000000, 3747.000000, 7271.000000, 4427.000000, 1849.000000, 3352.000000, 318.000000, 3508.000000, 2272.000000, 10129.000000, 10829.000000, 5116.000000, 2567.000000, 
1578.000000, 6237.000000, 989.000000, 8531.000000, 469.000000, 6691.000000, 1502.000000, 10305.000000, 7559.000000, 10900.000000, 11630.000000, 4216.000000, 10706.000000, 8369.000000, 3183.000000, 9266.000000, 11177.000000, 2421.000000, 3305.000000, 2495.000000, 10760.000000, 1992.000000, 4494.000000, 3514.000000, 2709.000000, 8561.000000, 1209.000000, 10993.000000, 9179.000000, 10532.000000, 
1263.000000, 5516.000000, 11765.000000, 8445.000000, 2693.000000, 1111.000000, 9296.000000, 3460.000000, 7372.000000, 7959.000000, 6955.000000, 12257.000000, 7242.000000, 9983.000000, 2468.000000, 6534.000000, 2259.000000, 9198.000000, 5764.000000, 7337.000000, 5300.000000, 3162.000000, 8472.000000, 4976.000000, 525.000000, 2903.000000, 5240.000000, 3828.000000, 7315.000000, 8395.000000, 
22.000000, 6187.000000, 5909.000000, 2705.000000, 2746.000000, 8692.000000, 6759.000000, 911.000000, 9111.000000, 8413.000000, 933.000000, 7938.000000, 8098.000000, 457.000000, 2745.000000, 2143.000000, 3913.000000, 648.000000, 6736.000000, 8181.000000, 5411.000000, 10779.000000, 10705.000000, 6067.000000, 9205.000000, 10743.000000, 10356.000000, 5927.000000, 11019.000000, 9355.000000, 
3733.000000, 11668.000000, 4808.000000, 3396.000000, 581.000000, 5516.000000, 3786.000000, 8937.000000, 9592.000000, 10958.000000, 684.000000, 9.000000, 11507.000000, 7949.000000, 825.000000, 992.000000, 2153.000000, 4397.000000, 7207.000000, 2613.000000, 2593.000000, 4230.000000, 3948.000000, 6263.000000, 3018.000000, 5583.000000, 6313.000000, 5092.000000, 5134.000000, 6779.000000, 
8149.000000, 9724.000000, 8852.000000, 5238.000000, 4434.000000, 6350.000000, 5796.000000, 1987.000000, 5284.000000, 2095.000000, 11782.000000, 3117.000000, 7591.000000, 11691.000000, 3998.000000, 4838.000000, 298.000000, 4268.000000, 9993.000000, 11225.000000, 11223.000000, 4244.000000, 9010.000000, 1279.000000, 9420.000000, 7818.000000, 7942.000000, 6627.000000, 2509.000000, 2456.000000, 
11550.000000, 3197.000000, 4337.000000, 12160.000000, 8541.000000, 8316.000000, 5590.000000, 11202.000000, 9465.000000, 3978.000000, 3672.000000, 2339.000000, 7691.000000, 5771.000000, 8381.000000, 8459.000000, 6209.000000, 1021.000000, 665.000000, 9588.000000, 5900.000000, 5966.000000, 7372.000000, 818.000000, 2956.000000, 6368.000000, 9836.000000, 3991.000000, 8691.000000, 7400.000000, 
2830.000000, 5539.000000, 7047.000000, 11858.000000, 10381.000000, 3689.000000, 7071.000000, 3433.000000, 4787.000000, 7316.000000, 11891.000000, 954.000000, 6932.000000, 432.000000, 6332.000000, 5622.000000, 5057.000000, 10670.000000, 690.000000, 4770.000000, 9917.000000, 5953.000000, 8858.000000, 8407.000000, 755.000000, 6660.000000, 753.000000, 6664.000000, 5068.000000, 3375.000000, 
1622.000000, 7096.000000, 3763.000000, 5873.000000, 1412.000000, 6214.000000, 10790.000000, 1338.000000, 11117.000000, 9322.000000, 8155.000000, 9179.000000, 8043.000000, 10635.000000, 9787.000000, 12253.000000, 2304.000000, 6153.000000, 384.000000, 7988.000000, 11871.000000, 4125.000000, 6461.000000, 1502.000000, 131.000000, 3969.000000, 10969.000000, 11105.000000, 11258.000000, 6726.000000, 
10100.000000, 5872.000000, 8513.000000, 5710.000000, 9680.000000, 1049.000000, 7926.000000, 582.000000, 6033.000000, 886.000000, 5772.000000, 4919.000000, 2867.000000, 6311.000000, 9953.000000, 9354.000000, 6924.000000, 193.000000, 5844.000000, 7729.000000, 5066.000000, 9664.000000, 11580.000000, 5521.000000, 455.000000, 11189.000000, 10801.000000, 1559.000000, 5754.000000, 4.000000, 
5265.000000, 915.000000, 5725.000000, 5430.000000, 10918.000000, 5851.000000, 6012.000000, 10330.000000, 2482.000000, 4025.000000, 6231.000000, 7545.000000, 11241.000000, 9675.000000, 1985.000000, 8639.000000, 5134.000000, 4303.000000, 12177.000000, 11175.000000, 7180.000000, 744.000000, 3070.000000, 5719.000000, 6604.000000, 11393.000000, 6518.000000, 2323.000000, 2386.000000, 5602.000000, 
3505.000000, 11607.000000, 9690.000000]
'''

In [ ]:
'''
# FFT
userma_buf_out= allocate(shape=(1024,), dtype=np.float64)
mmio_usedma.write(0x38, userma_buf_out.device_address)  # output buffer addr low
mmio_usedma.write(0x3c, 0x00000000) # outputbuffer addr high
userma_buf= allocate(shape=(1024,), dtype=np.float64)
mmio_usedma.write(0x44,userma_buf.device_address)  # input buffer addr low
mmio_usedma.write(0x48, 0x00000000) # inputbuffer addr high
mmio_usedma.write(0x10, 0x0000000) # mode

for i in range(1024):
    userma_buf[i]=FFT_in[i]

# dma_start
mmio_usedma.write(0x00,0x00000001) 
timeKernelStart = time()
while True:
    if mmio_usedma.read(0x18) == 0x01:
        break
timeKernelEnd = time()

# FFT output
for i in range(1024):
    print(userma_buf_out[i])

# Execution time
print("FFT execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
print(hex(userma_buf.device_address-userma_buf_out.device_address))
'''